In [1]:
import pandas as pd
data = pd.read_csv('cvrm_master.csv')
from cleaning_function import *
import numpy as np
pd.set_option('display.max_rows', 120)

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6528/3757516805.py:2: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


# Convert Numerics

In [2]:
data = drop_null_columns(data, 0.95)

In [3]:
df = data

percent_columns = [col for col in df.columns if "Percent" in col]

print("Columns to check:", percent_columns)

# Function to standardize percentage representation
def standardize_percentage_column(series):
    """Detects and converts percentage values into correct decimal representation."""
    series = series.astype(str).str.replace('%', '', regex=True)  # Remove '%' if present
    series = pd.to_numeric(series, errors='coerce')  # Convert to numeric

    if series.isna().all():
        return series  # Skip empty columns

    # Get the 5 highest values in the column
    max_5_values = series.nlargest(5).values
    
    # If all 5 max values are >1, assume whole number percentages and divide by 100
    if all(val > 1 for val in max_5_values):
        series = series / 100  

    return series

# Apply standardization to relevant columns
for col in df.columns:
    if "%" in col:  # Always divide columns with '%' in their names by 100
        df[col] = df[col].astype(str).str.replace('%', '', regex=True).astype(float) / 100
    elif col in percent_columns:  # Apply the rule-based check for numerical columns
        df[col] = standardize_percentage_column(df[col])

Columns to check: ['Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2016', 'Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2017', 'Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2018', 'Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2019', 'Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2020', 'Percent_Of_Days_With_Level_Of_PM2.5_Exceeding_The_Standard_EPA2021', 'Total_population_Percent_With_Daily_Commute_Less_Than_15min_ACS2017_21', 'Total_population_Percent_With_Daily_Commute_Between_15_60min_ACS2017_21', 'Total_population_Percent_With_Daily_Commute_60min_And_Above_ACS2017_21', 'Black_AA_Single_Race_Percent_With_Daily_Commute_Less_Than_15min_ACS2017_21', 'Black_AA_Single_Race_Percent_With_Daily_Commute_Between_15_60min_ACS2017_21', 'Black_AA_Single_Race_Percent_With_Daily_Commute_60min_And_Above_ACS2017_21', 'AIAN_Single_Race_Percent_With_Daily_Commute_Less_Than_15min_ACS2017_21', 'AIAN_Single_Race_Percent_Wit

In [4]:
column_name = 'Black_AA_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22'
if column_name in df.columns:
    non_numeric_values = df[column_name][~df[column_name].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()].unique()
    print(f"Non-numeric values in '{column_name}':", non_numeric_values)
else:
    print(f"Column '{column_name}' not found in the dataset.")

Non-numeric values in 'Black_AA_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22': [nan]


In [5]:
columns_to_check = [
    'Populations_Vulnerable_to_Predicted_Surface_Smoke_from_Wildland_Fires_NOAA2020',
    'Black_AA_Single_Race_Female_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Female_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Male_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Male_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Asian_Single_Race_Female_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Asian_Single_Race_Male_CVD_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Female_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Black_AA_Single_Race_Male_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Male_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Obesity_and_Hyperalimentation_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Male_Obesity_and_Hyperalimentation_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Number_of_hospitals_County_CDCnehTracking_2020',
    'Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20'
]

from collections import Counter
all_values = []

for col in columns_to_check:
    if col in df.columns:
        try:
            values = df[col].dropna().astype(str).tolist()
            all_values.extend(values)
        except Exception as e:
            print(f"Error processing column {col}: {e}")
value_counts = Counter(all_values)
top_10_common_values = value_counts.most_common(10)
top_10_df = pd.DataFrame(top_10_common_values, columns=["Value", "Count"])
top_10_df

,Value,Count
0,1.0,1476
1,2.0,452
2,3.0,167
3,4.0,97
4,5.0,61
5,6.0,51
6,15.0,45
7,8.0,44
8,7.0,42
9,24.3,41


In [6]:
replace_with_nan = ["Data Not Collected", "No Counts", "Not Available", "Unreliable"]

# Replace specified words/phrases with NaN in non-numeric columns
for col in columns_to_check:
    df[col] = df[col].replace(replace_with_nan, np.nan)

# Convert cleaned columns to numeric where possible
for col in columns_to_check:
    try:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except ValueError:
        pass  # Retain as object if still not numeric



KeyError: 'Populations_Vulnerable_to_Predicted_Surface_Smoke_from_Wildland_Fires_NOAA2020'

In [198]:
non_numeric = df.select_dtypes(exclude = [np.number])
non_numeric

,statecounty_x,mapped_match,stname,name,Code,State,County [2],County Seat(s) [3],Water Area km²,Water Area mi²,Latitude,Longitude
0,01autauga,01autauga,Alabama,Autauga,01autauga,AL,Autauga,Prattville,25.776,9.952,+32.536382°,–86.644490°
1,01baldwin,01baldwin,Alabama,Baldwin,01baldwin,AL,Baldwin,Bay Minette,1133.19,437.527,+30.659218°,–87.746067°
2,01barbour,01barbour,Alabama,Barbour,01barbour,AL,Barbour,Clayton,50.865,19.639,+31.870670°,–85.405456°
3,01bibb,01bibb,Alabama,Bibb,01bibb,AL,Bibb,Centreville,9.289,3.587,+33.015893°,–87.127148°
4,01blount,01blount,Alabama,Blount,01blount,AL,Blount,Oneonta,15.157,5.852,+33.977448°,–86.567246°
...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56sweetwater,56sweetwater,Wyoming,Sweetwater,56sweetwater,WY,Sweetwater,Green River,166.887,64.436,+41.660339°,–108.875676°
3139,56teton,56teton,Wyoming,Teton,56teton,WY,Teton,Jackson,572.266,220.953,+44.049321°,–110.588102°
3140,56uinta,56uinta,Wyoming,Uinta,56uinta,WY,Uinta,Evanston,16.342,6.31,+41.284726°,–110.558947°
3141,56washakie,56washakie,Wyoming,Washakie,56washakie,WY,Washakie,Worland,10.762,4.155,+43.878831°,–107.669052°


In [199]:
columns_to_clean = ['Latitude', 'Longitude']
for col in columns_to_clean:
    df[col] = (
        df[col]
        .str.replace('°', '', regex=False)  # Remove degree symbol
        .str.replace('+', '', regex=False)  # Remove '+' signs
        .str.replace('−', '-', regex=False)  # Replace special minus sign (Unicode) with standard '-'
        .str.replace('–', '-', regex=False)  # Replace alternative special minus sign (Unicode) with standard '-'
    )
    df[col] = pd.to_numeric(df[col])



In [200]:
# df.to_csv('cvrm_master.csv', index = False)

# FAISS

In [201]:
cvrm_keywords = [
    "heart", "cardio", "stroke", "hypertension", "diabetes", "renal", "kidney", 
    "cholesterol", "blood pressure", "glucose", "insulin", "lipid", "triglyceride", 'cvd'
]
numeric_columns = df.select_dtypes(include=[np.number]).columns
cvrm_columns = [col for col in numeric_columns if any(keyword in col for keyword in cvrm_keywords)]
print("CVRM-related numeric columns:", cvrm_columns)


CVRM-related numeric columns: ['HealthOutcome_percent_pop_with_highcholesterol_PLACES_BRFSS_2021', 'HealthOutcome_percent_pop_with_stroke_PLACES_BRFSS_2021', 'HealthOutcome_percent_pop_with_chronic_kidney_diseases_Places_BRFSS_2021', 'HealthOutcome_percent_pop_with_hypertension_PLACES_BRFSS_2021', 'HealthOutcome_percent_pop_with_coronary_heart_diseases_PLACES_BRFSS_2021', 'percent_pop_with_cholesterol_screening_Places_BRFSS_2021', 'percent_pop_taking_medicacine_for_hypertension_control_Places_BRFSS_2021']


In [202]:
import faiss
cvrm_data = df[cvrm_columns].dropna().to_numpy().astype('float32')
index = faiss.IndexFlatL2(cvrm_data.shape[1])
index.add(cvrm_data)
query_vector = cvrm_data[0].reshape(1, -1)
k = 5
distances, indices = index.search(query_vector, k)
print("Distances:", distances)
print("Indices of closest matches:", indices)

Distances: [[0.        1.900011  2.0600028 2.189995  2.199999 ]]
Indices of closest matches: [[   0 1371   18   33 2809]]


In [203]:
df[cvrm_columns].isnull().sum()

HealthOutcome_percent_pop_with_highcholesterol_PLACES_BRFSS_2021            67
HealthOutcome_percent_pop_with_stroke_PLACES_BRFSS_2021                     67
HealthOutcome_percent_pop_with_chronic_kidney_diseases_Places_BRFSS_2021    67
HealthOutcome_percent_pop_with_hypertension_PLACES_BRFSS_2021               67
HealthOutcome_percent_pop_with_coronary_heart_diseases_PLACES_BRFSS_2021    67
percent_pop_with_cholesterol_screening_Places_BRFSS_2021                    67
percent_pop_taking_medicacine_for_hypertension_control_Places_BRFSS_2021    67
dtype: int64

In [204]:
renal_col = include_columns(df, ['renal'])
renal_col

,Black_AA_Single_Race_Female_Percent_of_Renal_Failure_In_Total_Deaths_CDC2018_22_x,Black_AA_Single_Race_Male_Percent_of_Renal_Failure_In_Total_Deaths_CDC2018_22_x,White_Single_Race_Female_Percent_of_Renal_Failure_In_Total_Deaths_CDC2018_22_x,White_Single_Race_Male_Percent_of_Renal_Failure_In_Total_Deaths_CDC2018_22_x,"Black_AA_Single_Race_Female_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22","Black_AA_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22","White_Single_Race_Female_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22","White_Single_Race_Male_Renal_Failure_Deaths_Rate_Per_100,000_Residents_CDC2018_22",Black_AA_Single_Race_Female_Renal_Failure_Deaths_Counts_CDC2018_22_x,Black_AA_Single_Race_Male_Renal_Failure_Deaths_Counts_CDC2018_22_x,White_Single_Race_Female_Renal_Failure_Deaths_Counts_CDC2018_22_x,White_Single_Race_Male_Renal_Failure_Deaths_Counts_CDC2018_22_x,Asian_Single_Race_Female_Renal_Failure_Deaths_Counts_CDC2018_22_x,Asian_Single_Race_Male_Renal_Failure_Deaths_Counts_CDC2018_22_x,NHPI_Single_Race_Female_Renal_Failure_Deaths_Counts_CDC2018_22_x,NHPI_Single_Race_Male_Renal_Failure_Deaths_Counts_CDC2018_22_x
0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,23.4,NaN,11.0,17.0,24.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,NaN,50.5,16.0,23.6,12.0,24.0,79.0,110.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,16.0,16.0,15.0,NaN,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,NaN,NaN,0.0,0.0,NaN,NaN,18.6,22.6,NaN,NaN,24.0,28.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
columns_with_2020_2021 = [col for col in df.columns if '2020' in col or '2021' in col or '2019' in col]
len(columns_with_2020_2021)
# no single race, 


295

In [210]:
columns_single = [col for col in df.columns if 'Single_Race' in col]
len(columns_single)

1099

# GraphSage

In [2]:
data = drop_null_columns(data, 0.6)

In [3]:
import torch
from sklearn.decomposition import PCA

In [4]:
data_df = data
data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Handle non-numeric data
categorical_cols = data_df.select_dtypes(include=['object']).columns
data_df[categorical_cols] = data_df[categorical_cols].fillna('missing').astype('category').apply(lambda x: x.cat.codes)

# Handle missing values
missing_indicator = data_df.isna().astype(int)  # Create missing value mask
data_df = data_df.fillna(0)  # Fill missing values with 0

# Optional: Reduce dimensionality with PCA
pca = PCA(n_components=20)  # Reduce to 20 dimensions
reduced_features = pca.fit_transform(data_df.values)

# Convert to PyTorch tensor
features_tensor = torch.tensor(reduced_features, dtype=torch.float)

In [5]:
from sklearn.neighbors import kneighbors_graph

# Create a KNN graph (connect each node to its 10 nearest neighbors)
adjacency_matrix = kneighbors_graph(features_tensor.numpy(), n_neighbors=10, mode='connectivity')
edge_index = torch.tensor(adjacency_matrix.nonzero(), dtype=torch.long)


/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6528/2652524758.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  edge_index = torch.tensor(adjacency_matrix.nonzero(), dtype=torch.long)


In [6]:
from torch_geometric.data import Data

# Create the graph object
graph_data = Data(x=features_tensor, edge_index=edge_index)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/libpyg.so, 0x0006): Symbol not found: __ZN5torch8autograd12VariableInfoC1ERKN2at6TensorE
  Referenced from: <E1B15A5F-A06F-32B5-96C2-8341AB4BC6D0> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/libpyg.so
  Expected in:     <E3D17B4A-4867-3D49-BC92-E04C28EE0F45> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "


In [7]:
from torch_geometric.nn import SAGEConv
import torch.nn as nn

class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model
input_dim = features_tensor.shape[1]
hidden_dim = 64
output_dim = features_tensor.shape[1]
model = GraphSAGEModel(input_dim, hidden_dim, output_dim)


In [8]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# 🛠 Fix: Ensure all missing values (NaN, Inf) are handled properly
features_tensor[torch.isnan(features_tensor)] = 0  # Replace NaNs with 0
features_tensor[features_tensor == float("inf")] = 0  # Replace Inf with 0
features_tensor[features_tensor == float("-inf")] = 0  # Replace -Inf with 0

# 🚀 Standardize Features for Numerical Stability
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_tensor.numpy())  # Standardize to mean=0, std=1
features_tensor = torch.tensor(scaled_features, dtype=torch.float)

batch_size = 256  # Reduce batch size if needed
num_nodes = graph_data.x.shape[0]

# 🏋️ Improved Optimizer with Weight Decay (Prevents Overfitting)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Lowered LR & added weight decay
criterion = nn.MSELoss()

# 🎯 Model Training with Gradient Clipping
model.train()
for epoch in range(10):
    sampled_nodes = random.sample(range(num_nodes), batch_size)
    sampled_nodes = torch.tensor(sampled_nodes, dtype=torch.long)

    # 🛠 Fix: Ensure `edge_index` does not contain nodes outside the batch
    edge_mask = torch.isin(graph_data.edge_index[0], sampled_nodes) & torch.isin(graph_data.edge_index[1], sampled_nodes)
    subgraph_edges = graph_data.edge_index[:, edge_mask]

    # 🛠 Fix: Remap edges to local batch indices
    node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(sampled_nodes)}
    remapped_edges = torch.tensor(
        [[node_mapping[n.item()] for n in subgraph_edges[i]] for i in range(2)], dtype=torch.long
    )

    # Extract subgraph features
    subgraph_x = features_tensor[sampled_nodes]

    optimizer.zero_grad()
    out = model(subgraph_x, remapped_edges)  # Train on subsampled nodes

    # ✅ Fix: Ensure `mask` aligns with the selected features
    mask = torch.tensor(
        missing_indicator.values[sampled_nodes, :features_tensor.shape[1]] == 1, dtype=torch.bool
    )

    # 🏋️ Apply loss only on missing values
    epsilon = 1e-9
    loss = criterion(out[mask] + epsilon, features_tensor[sampled_nodes][mask] + epsilon)


    loss.backward()

    # 🚀 Apply Gradient Clipping to Prevent Exploding Gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.015017181634902954
Epoch 2, Loss: 0.03788028657436371
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan


In [9]:
model.eval()
with torch.no_grad():
    imputed_features = model(features_tensor, graph_data.edge_index)
final_features = features_tensor.clone()
nan_mask = torch.isnan(features_tensor)
final_features[nan_mask] = imputed_features[nan_mask]

In [10]:
mae = torch.abs((imputed_features - features_tensor)[nan_mask]).mean()
print(f"MAE of Imputed Features: {mae.item():.4f}")


MAE of Imputed Features: nan


In [11]:
model.eval()
with torch.no_grad():
    imputed_features = model(features_tensor, graph_data.edge_index)


In [12]:
imputed_array = imputed_features.numpy()

data_df_filled = data_df.copy() 
for i, col in enumerate(data_df.columns):
    if data_df[col].isna().any():
        data_df_filled.loc[data_df[col].isna(), col] = imputed_array[data_df[col].isna(), i]


In [13]:
data_df_filled

,statecounty_x,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,...,County Seat(s) [3],Population (2010),Land Area km²,Land Area mi²,Water Area km²,Water Area mi²,Total Area km²,Total Area mi²,Latitude,Longitude
0,0,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,...,1694,54571,1539.582,594.436,1047,1010,1565.358,604.388,32.536382,-86.644490
1,1,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,...,145,182265,4117.522,1589.784,1983,1938,5250.712,2027.311,30.659218,-87.746067
2,2,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,...,413,27457,2291.819,884.876,1399,1357,2342.684,904.515,31.870670,-85.405456
3,3,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,...,351,22915,1612.481,622.582,511,492,1621.770,626.169,33.015893,-87.127148
4,4,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,...,1539,57322,1669.962,644.776,753,726,1685.119,650.628,33.977448,-86.567246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3138,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,...,849,43806,27004.897,10426.649,1748,1703,27171.784,10491.085,41.660339,-108.875676
3139,3139,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,...,1026,21294,10347.984,3995.379,1916,1871,10920.250,4216.332,44.049321,-110.588102
3140,3140,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,...,681,21118,5390.450,2081.264,799,769,5406.791,2087.574,41.284726,-110.558947
3141,3141,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,...,2282,8533,5797.815,2238.549,571,549,5808.577,2242.704,43.878831,-107.669052


In [14]:
data_df_filled.isnull().sum().sum()

0

In [16]:
data_df_filled.to_csv('graphsage_df.csv', index=False)